In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import create_model
import inspect, json
from inspect import Parameter



In [ ]:

print(load_dotenv())

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

### Formating a Custom Function or Tool

In [1]:
# Doc string helps AI to call the relevant tool/function

def custom_fn(num1: int , num2: int ) -> int:

    # Compute custom_fn between two numbers
    return (num1 + num2)^2

print(custom_fn(3, 4))


5


#### To Convert Python Fn Code to JSON Format

In [2]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()

    return dict(name=f.__name__, description=f.__doc__, parameters=s)



In [7]:
custom_fn_json = jsonschema(custom_fn)
custom_fn_json

C:\Users\preet\AppData\Local\Temp\ipykernel_15688\1043263122.py:14: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'custom_fn',
 'description': None,
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `custom_fn`',
  'type': 'object'}}

## Create Agent

In [ ]:
model_name = "gpt-3.5-turbo"

client = OpenAI()
   
response = client.chat.completions.create(
  model= model_name,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "compute abc between 2 and 3"},
  ],
)

response.choices[0].message.content


#### Passing tool to AI Model

In [ ]:
messages= [
    {"role": "user", "content": "Compute abc between 2 and 3"}
]

# Pass th function to GPT model
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    functions=[custom_fn_json],
    function_call="auto",
    temperature=0
)
response


In [ ]:
print(response.choices[0].message.function_call)
print(response.choices[0].message.function_call.arguments)
print(type(response.choices[0].message.function_call.arguments))

In [ ]:
func_name = response.choices[0].message.function_call.name
func_args = json.loads(response.choices[0].message.function_call.arguments)
print("Function name:", func_name)
print("Function arguments:", func_args)
print(type(func_args))

In [ ]:
if func_name == 'custom_fn':
    result = custom_fn(**func_args)
print(result)

### Using Langchain

In [ ]:
from langchain_core.tools import tool

@tool
def custom_fn(num1: int , num2: int ) -> int:
    # Compute custom_fn between two numbers
    return (num1 + num2)^2


print( custom_fn.description )


In [ ]:

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [custom_fn]

llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke("Compute abc between 2 and 3")
response.additional_kwargs
